### Import required libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertTokenizerFast
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,  precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
import time

### Example Sentences

In [ ]:
tricky_sentences = [
...    "Most automated sentiment analysis tools are shit.",
...    "VADER sentiment analysis is amazingly bad.",
...    "Sentiment analysis has never been good.",
...    "Sentiment analysis with VADER has never been this good.",
...    "Warren Beatty has never been so entertaining.",
...    "I won't say that the movie is astounding and I wouldn't claim that \
...    the movie is too banal either.",
...    "I like to hate Michael Bay films, but I couldn't fault this one",
...    "I like to hate Michael Bay films, BUT I couldn't help but fault this one",
...    "It's one thing to watch an Uwe Boll film, but another thing entirely \
...    to pay for it",
...    "The movie was too good",
...    "This movie was actually neither that funny, nor super witty.",
...    "This movie doesn't care about cleverness, wit or any other kind of \
...    intelligent humor.",
...    "Those who find ugly meanings in beautiful things are corrupt without \
...    being charming.",
...    "There are slow and repetitive parts, BUT it has just enough spice to \
...    keep it interesting.",
...    "The script is not fantastic, but the acting is decent and the cinematography \
...    is EXCELLENT!",
...    "Roger Dodger is one of the most compelling variations on this theme.",
...    "Roger Dodger is one of the least compelling variations on this theme.",
...    "Roger Dodger is at least compelling as a variation on the theme.",
...    "they fall in love with the product",
...    "but then it breaks",
...    "usually around the time the 90 day warranty expires",
...    "the twin towers collapsed today",
...    "However, Mr. Carter solemnly argues, his client carried out the kidnapping \
...    under orders and in the ''least offensive way possible.''"
... ]

# My Implementation of DistilBert

In [ ]:
tokenizer_1 = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model_1 = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
def sentiment_score_1(review):
    tokens = tokenizer_1.encode(review, return_tensors='pt')
    result = model_1(tokens)
    prediction = int(torch.argmax(result.logits))

    print(result)
    print(prediction)
    # Classify sentiment
    if prediction == 1:
        sentiment_label = "Positive"
    else:
        sentiment_label = "Negative"

    return sentiment_label # return logits as well?

In [ ]:
for sentence in tricky_sentences:

    sentiment_label = sentiment_score_1(sentence)

    print("Sentence:", sentence)
    print("Sentiment:", sentiment_label)
    print()

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.6491, -3.7305]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0
Sentence: Most automated sentiment analysis tools are shit.
Sentiment: Negative

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.7243, -3.7802]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0
Sentence: VADER sentiment analysis is amazingly bad.
Sentiment: Negative

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.6214, -3.6775]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0
Sentence: Sentiment analysis has never been good.
Sentiment: Negative

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.1775, -0.8812]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0
Sentence: Sentiment analysis with VADER has never been this good.
Sentiment: Negative

SequenceClassifierOutput(loss=None, logits=tensor([[-3.9581,  4.1617]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
sentiment_score_1("hello, what a bad morning!")


SequenceClassifierOutput(loss=None, logits=tensor([[ 3.1201, -2.5089]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0


'Negative'

# Small Sample Finetuning of Distilbert
**the code below is just my efforts for a working code, and a check for presence of accuracy metrics' increases**

### Establishing a baseline for comparison

In [ ]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Load validation data
val_data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_validate_100.csv')
val_texts = val_data['Review'].tolist()
val_labels = val_data['Rating'].tolist()

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

# Predict with the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Confusion matrix:\n {conf_matrix}')

# Generate and print classification report
class_names = ['0', '1']
report = classification_report(true_labels, predictions, target_names=class_names)
print('Classification Report:')
print(report)

Accuracy: 0.45
F1-score: 0.6099290780141844
Precision: 0.4725274725274725
Recall: 0.86
Confusion matrix:
 [[ 2 48]
 [ 7 43]]
Classification Report:
              precision    recall  f1-score   support

           0       0.22      0.04      0.07        50
           1       0.47      0.86      0.61        50

    accuracy                           0.45       100
   macro avg       0.35      0.45      0.34       100
weighted avg       0.35      0.45      0.34       100



### Actual finetuning process

In [ ]:
# Record start time
start_time = time.time()

# Load data
data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_TRAIN_1000.csv')
reviews = data['Review'].tolist()
labels = data['Rating'].tolist()  # assuming sentiment is encoded as 0 (negative) and 1 (positive)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2)

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloaders
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model
model_path = "/content/drive/MyDrive/CZ4034/sentiment_model_DistilBERT"  # Specify the directory where you want to save the model
model.save_pretrained(model_path)

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Time required to fine-tune:  37.17642521858215


### **Time to see how our fine-tuned model does on the unseen validation dataset.**

In [ ]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/CZ4034/sentiment_model_DistilBERT')
model = model.to('cuda')  # if GPU is available

# Load validation data
val_data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_validate_100.csv')
val_texts = val_data['Review'].tolist()
val_labels = val_data['Rating'].tolist()

# Initialize tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:

# Evaluate the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Confusion matrix:\n {conf_matrix}')

# Generate and print classification report
class_names = ['0', '1']
report = classification_report(true_labels, predictions, target_names=class_names)
print('Classification Report:')
print(report)

# Observe what the results actually are
print("True labels:", true_labels)
print("Predicted labels:", predictions)

Accuracy: 1.0
F1-score: 1.0
Precision: 1.0
Recall: 1.0
Confusion matrix:
 [[50  0]
 [ 0 50]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        50

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

True labels: [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Predicted labels: [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,

# **Actual Finetuning of DistilBert with project datasets**
**The code below uses our project's full-sized dataset, and external training datasets.**

### Establishing a baseline for comparison

In [ ]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Load validation data
val_data = pd.read_csv('/content/drive/MyDrive/CZ4034/evaluation_data_raw_final.csv')
val_texts = val_data['Overall Review with Title'].tolist()
val_labels = val_data['Sentiment'].map({'Positive': 1, 'Negative': 0}).tolist()  # convert sentiment to numeric

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:

# Predict with the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

# Generate and print classification report
class_names = ['0', '1']
report = classification_report(true_labels, predictions, target_names=class_names)
print('Classification Report:')
print(report)

Accuracy: 0.8346867749419954
Precision: 0.8998609179415855
Recall: 0.7523255813953489
F1-score: 0.8195060164661179
Confusion matrix:
 [[792  72]
 [213 647]]
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       864
           1       0.90      0.75      0.82       860

    accuracy                           0.83      1724
   macro avg       0.84      0.83      0.83      1724
weighted avg       0.84      0.83      0.83      1724



### Actual finetuning process

In [ ]:
# Record start time
start_time = time.time()

# Load data
data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_TRAIN_balanced_final_36000.csv')
reviews = data['Review'].tolist()
labels = data['Rating'].tolist()  # assuming sentiment is already encoded as 0 (negative) and 1 (positive)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2)

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloaders
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model
model_save_path = "/content/drive/MyDrive/CZ4034/sentiment_model_DistilBERT_CZ4034"  # Specify the directory where you want to save the model
model.save_pretrained(model_save_path)

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

Time required to fine-tune:  1271.413997888565


### **Time to see how our fine-tuned model does on the unseen validation dataset.**

In [ ]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained(model_save_path) # Specify the path where you saved the model
model = model.to('cuda')  # if GPU is available

# Load validation data
val_data = pd.read_csv('/content/drive/MyDrive/CZ4034/evaluation_data_raw_final.csv')
val_texts = val_data['Overall Review with Title'].tolist()
val_labels = val_data['Sentiment'].map({'Positive': 1, 'Negative': 0}).tolist()  # convert sentiment to numeric

# Initialize tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Record start time
start_time = time.time()

# Evaluate the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Record end time and thus time metrics
end_time = time.time()
print("Time required to classify records: ", end_time - start_time)
print("Records classified per second: ", len(val_labels)/(end_time - start_time))

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

# Generate and print classification report
class_names = ['0', '1']
report = classification_report(true_labels, predictions, target_names=class_names)
print('Classification Report:')
print(report)

Time required to classify records:  8.226654767990112
Records classified per second:  209.56270180536546
Accuracy: 0.9135730858468677
Precision: 0.8868335146898803
Recall: 0.9476744186046512
F1-score: 0.9162450815064643
Confusion matrix:
 [[760 104]
 [ 45 815]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.88      0.91       864
           1       0.89      0.95      0.92       860

    accuracy                           0.91      1724
   macro avg       0.92      0.91      0.91      1724
weighted avg       0.92      0.91      0.91      1724



### Append lists to dataframe and export as csv

In [ ]:
val_data['True_Labels'] = true_labels # Validate against Sentiment column, should have matching 1-Positive, 0-Negative
val_data['Predicted_Labels'] = predictions
val_data.to_csv('/content/drive/MyDrive/CZ4034/validation_results.csv', index=False)

In [ ]:
val_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1724 entries, 0 to 1723
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Name               1724 non-null   object
 1   Overall Rating             1724 non-null   int64 
 2   Review Date                1724 non-null   object
 3   Review Title               1724 non-null   object
 4   Job Title                  1724 non-null   object
 5   Job Details                1724 non-null   object
 6   Location                   1724 non-null   object
 7   Pros                       1724 non-null   object
 8   Cons                       1724 non-null   object
 9   Overall Review             1724 non-null   object
 10  Overall Review with Title  1724 non-null   object
 11  Sentiment                  1724 non-null   object
 12  True_Labels                1724 non-null   int64 
 13  Predicted_Labels           1724 non-null   int64 
dtypes: int64

## **Executing on final cleaned dataset**

In [ ]:
tokenizer_finetuned = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model_finetuned = DistilBertForSequenceClassification.from_pretrained(model_save_path) # Specify the path where you saved the model


In [ ]:
def sentiment_score_finetuned(review):
    tokens = tokenizer_finetuned.encode(review, return_tensors='pt')
    result = model_finetuned(tokens)
    prediction = int(torch.argmax(result.logits))

    # Classify sentiment
    if prediction == 1:
        sentiment_label = "Positive"
    else:
        sentiment_label = "Negative"

    return sentiment_label # return logits as well?

In [ ]:
# Just to check if the function has been properly defined
sentiment_score_finetuned('This is a good sentence.')

'Positive'

In [ ]:
# Import dataset, using first column as index column
glassdoor_data_distilbert_finetuned = pd.read_csv('/content/drive/MyDrive/CZ4034/final_sg_companies_reviews_clean_UID.csv') # Change this to the new clean one, concated alr?

# Concatenating Review Title, Pros and Cons
# Unsure if this is the method used to concat the columns
glassdoor_data_distilbert_finetuned['Overall Review with Title'] = glassdoor_data_distilbert_finetuned['Review Title'] + ' // ' + glassdoor_data_distilbert_finetuned['Pros'] + ' ' + glassdoor_data_distilbert_finetuned['Cons']

In [ ]:
glassdoor_data_distilbert_finetuned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18349 entries, 0 to 18348
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18349 non-null  object 
 1   Company Name               18349 non-null  object 
 2   Overall Rating             18349 non-null  float64
 3   Review Date                18349 non-null  object 
 4   Review Title               18349 non-null  object 
 5   Job Title                  18342 non-null  object 
 6   Job Details                18349 non-null  object 
 7   Location                   13005 non-null  object 
 8   Pros                       18349 non-null  object 
 9   Cons                       18349 non-null  object 
 10  Overall Review with Title  18349 non-null  object 
dtypes: float64(1), object(10)
memory usage: 1.5+ MB


In [ ]:
#glassdoor_data_distilbert_finetuned["Pro's Sentiment"] = glassdoor_data_distilbert_finetuned['Pros'].apply(lambda x: sentiment_score_finetuned(x[:512]))
#glassdoor_data_distilbert_finetuned["Cons's Sentiment"] = glassdoor_data_distilbert_finetuned['Cons'].apply(lambda x: sentiment_score_finetuned(x[:512]))
glassdoor_data_distilbert_finetuned['Predicted Sentiment'] = glassdoor_data_distilbert_finetuned['Overall Review with Title'].apply(lambda x: sentiment_score_finetuned(x[:512]))
glassdoor_data_distilbert_finetuned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18349 entries, 0 to 18348
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18349 non-null  object 
 1   Company Name               18349 non-null  object 
 2   Overall Rating             18349 non-null  float64
 3   Review Date                18349 non-null  object 
 4   Review Title               18349 non-null  object 
 5   Job Title                  18342 non-null  object 
 6   Job Details                18349 non-null  object 
 7   Location                   13005 non-null  object 
 8   Pros                       18349 non-null  object 
 9   Cons                       18349 non-null  object 
 10  Overall Review with Title  18349 non-null  object 
 11  Predicted Sentiment        18349 non-null  object 
dtypes: float64(1), object(11)
memory usage: 1.7+ MB


In [ ]:
# Export DataFrame to CSV
glassdoor_data_distilbert_finetuned.to_csv('/content/drive/MyDrive/CZ4034/glassdoor_data_distilbert_finetuned.csv', index=False)